# Práctica 4
El problema que vamos a tratar de solucionar consiste en encontrar las estaciones con mayor número de bicicletas de salida y entrada.

Para implementar una solución, lo primero que vamos a hacer será llevar a cabo los diferentes pasos necesarios para conseguir dicho objetivo en un fichero de datos de prueba, como los que hemos usado en los ejemplos de clase.

In [1]:
from pyspark import SparkContext
sc = SparkContext()

23/05/20 14:01:06 WARN Utils: Your hostname, prpa resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/05/20 14:01:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/20 14:01:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
import json

In [3]:
rdd_base = sc.textFile('sample_10e3.json') #creamos un rdd a partir del fichero de datos.
rdd_base.take(10)

['{ "_id" : { "$oid" : "5cf83b752f3843a016be4e2f" }, "user_day_code" : "e4d55deb9ac172a8d8f5f0a32599815bd51b7c8760d67e42b11adf7c0829341b", "idplug_base" : 21, "user_type" : 1, "idunplug_base" : 8, "travel_time" : 219, "idunplug_station" : 90, "ageRange" : 0, "idplug_station" : 66, "unplug_hourTime" : { "$date" : "2019-06-01T00:00:00.000+0200" }, "zip_code" : "" }',
 '{ "_id" : { "$oid" : "5cf83b762f3843a016be4e48" }, "user_day_code" : "8a0c4123e924a50a958f51985eb71aea750fb072438035f149283490cc6bfaf4", "idplug_base" : 19, "user_type" : 1, "idunplug_base" : 19, "travel_time" : 359, "idunplug_station" : 71, "ageRange" : 4, "idplug_station" : 136, "unplug_hourTime" : { "$date" : "2019-06-01T00:00:00.000+0200" }, "zip_code" : "28039" }',
 '{ "_id" : { "$oid" : "5cf83b762f3843a016be4e4f" }, "user_day_code" : "a6a9c1f74a68496000542210abc4fc2eba79e2756ad5355a626632f7783dd418", "idplug_base" : 17, "user_type" : 1, "idunplug_base" : 7, "travel_time" : 375, "idunplug_station" : 39, "ageRange" : 4

In [4]:
#para colocarlo de una manera más cómoda de manipular, utilizamos la función mapper de los ejemplos.
def mapper(line):
    data = json.loads(line)
    u_t = data['user_type']
    u_c = data['user_day_code']
    start = data['idunplug_station']
    end = data['idplug_station']
    time = data['travel_time']
    return u_t, u_c, start, end, time
#y creamos el rdd aplicando la función al base
rdd_util = rdd_base.map(mapper)

In [5]:
rdd_util.take(10)

[(1,
  'e4d55deb9ac172a8d8f5f0a32599815bd51b7c8760d67e42b11adf7c0829341b',
  90,
  66,
  219),
 (1,
  '8a0c4123e924a50a958f51985eb71aea750fb072438035f149283490cc6bfaf4',
  71,
  136,
  359),
 (1,
  'a6a9c1f74a68496000542210abc4fc2eba79e2756ad5355a626632f7783dd418',
  39,
  38,
  375),
 (1,
  '5706c0bd494acc02279d532821c9666b0e506d4f81c838ef3147b2e0abbe7156',
  66,
  90,
  264),
 (1,
  'eb1b6d32bd4add5d5ff91af72a38786d61075c090383a53882f33b0801eb0079',
  152,
  166,
  367),
 (1,
  'c2905f6038aa9523da6bc3222c63c2ec3269c69281bf34eac161eed1ae2ba5f4',
  55,
  53,
  174),
 (1,
  'f94420744ea060ac42ef3901cb251bb5b4ff63033be7bcb96d8764b968912691',
  133,
  129,
  308),
 (1,
  'e75471af2ea032a327f636e28b8603c4cddab9155948fca85e82ec2f73c2bd1a',
  153,
  169,
  462),
 (1,
  'c0d73ee753773a3bfcc1483366a6ee68be9524dafbf65e81d31f4509d73304ac',
  44,
  129,
  482),
 (1,
  '47ecd557f21ca7aee6235dc015c6eaa7cea5e99e3aa8fbdfac8a308bb093f552',
  85,
  133,
  480)]

In [6]:
#filtramos los datos para quedarnos solo con los de user_type = 1 y transformamos cada dato en una tupla con la estación de salida, de entrada, y el tiempo de uso. 
rdd_users = rdd_util.filter(lambda x: x[0]==1).map(lambda x: (tuple(x[2:])))
rdd_users.take(2)

[(90, 66, 219), (71, 136, 359)]

In [7]:
#Primero contamos el número de bicis que salen de cada estación.
count_out = rdd_users.map(lambda x: x[0]).countByValue()
rdd_out = sc.parallelize(list(count_out.items())) #creamos un rdd con los valores obtenidos en count_out

#Después hacemos lo mismo con las estaciones a las que entran las bicis (por eso nos quedamos con el segundo elemento de cada tupla)
count_in = rdd_users.map(lambda x: x[1]).countByValue()
rdd_in = sc.parallelize(list(count_in.items()))
rdd_out.take(10)

[(90, 14),
 (71, 6),
 (39, 7),
 (66, 3),
 (152, 1),
 (55, 14),
 (133, 7),
 (153, 4),
 (44, 12),
 (85, 3)]

In [8]:
rdd_in.take(10)

[(66, 2),
 (136, 21),
 (38, 11),
 (90, 22),
 (166, 5),
 (53, 8),
 (129, 25),
 (169, 9),
 (133, 8),
 (83, 19)]

In [10]:
def agrupar_estaciones(par): #queremos meter cada estación en un grupo (las agruparemos de 10 en 10, siendo el grupo 0 las que van del 0-9, el 1 el que va del 10-29 y asi sucesivamente)
    clave = par[0]
    valor = par[1]
    grupo = clave // 10
    return grupo,valor

#aplicamos esta función a cada uno de los pares, tanto en el caso de rdd_out como en el rdd_in y después sumaremos cada uno de los valores de los grupos, para ver cuántas bicis de cada grupo de estaciones salen y entran.
rdd_out_agrup = rdd_out.map(agrupar_estaciones).reduceByKey(lambda x,y: x + y)
rdd_out_agrup.sortByKey().take(21)

[(0, 76),
 (1, 68),
 (2, 38),
 (3, 49),
 (4, 77),
 (5, 114),
 (6, 54),
 (7, 29),
 (8, 44),
 (9, 46),
 (10, 44),
 (11, 38),
 (12, 45),
 (13, 68),
 (14, 29),
 (15, 30),
 (16, 74),
 (17, 27),
 (19, 2),
 (21, 2)]

In [11]:
#hacemos lo misom con rdd_in
rdd_in_agrup = rdd_in.map(agrupar_estaciones).reduceByKey(lambda x,y: x + y)
rdd_in_agrup.sortByKey().take(21)

[(0, 39),
 (1, 46),
 (2, 26),
 (3, 27),
 (4, 74),
 (5, 92),
 (6, 31),
 (7, 58),
 (8, 70),
 (9, 47),
 (10, 27),
 (11, 34),
 (12, 92),
 (13, 99),
 (14, 45),
 (15, 42),
 (16, 65),
 (17, 33),
 (19, 5),
 (21, 2)]

In [12]:
#como lo que queremos ver es de qué grupo (de qué zona) salen más bicicletas y a qué zona llegan más bicicletas (de momento en un mes),
#lo ordenamos por valores, para ver como están de mayor a menor.
rdd_out_agrup.sortBy(lambda x : x[1], ascending = False).take(20)

[(5, 114),
 (4, 77),
 (0, 76),
 (16, 74),
 (13, 68),
 (1, 68),
 (6, 54),
 (3, 49),
 (9, 46),
 (12, 45),
 (8, 44),
 (10, 44),
 (11, 38),
 (2, 38),
 (15, 30),
 (7, 29),
 (14, 29),
 (17, 27),
 (19, 2),
 (21, 2)]

In [13]:
rdd_in_agrup.sortBy(lambda x : x[1], ascending = False).take(20)

[(13, 99),
 (5, 92),
 (12, 92),
 (4, 74),
 (8, 70),
 (16, 65),
 (7, 58),
 (9, 47),
 (1, 46),
 (14, 45),
 (15, 42),
 (0, 39),
 (11, 34),
 (17, 33),
 (6, 31),
 (3, 27),
 (10, 27),
 (2, 26),
 (19, 5),
 (21, 2)]

# Solución al problema
Ahora ya estamos en condiciones de poder aplicar las técnicas probadas anteriormente sobre un fichero de prueba, a todos los ficheros de datos correspondientes con cada mes del año.

Para ello, leeremos los ficheros de datos, agrupándolos en listas de tres en tres, correspondiéndose cada una de ellas con una época del año.

Después, tranformaremos todo esto en cuatro rdds y, poco a poco, implementando las funciones necesarias, obtendremos los resultados deseados.

Veamos con más detalle cómo:

In [34]:
#creamos, como se anticipó, las cuatro listas con los ficheros de datos correspondientes:

invierno = ["201801_Usage_Bicimad.json", "201802_Usage_Bicimad.json","201803_Usage_Bicimad.json"]
primavera = ["201804_Usage_Bicimad.json", "201805_Usage_Bicimad.json","201806_Usage_Bicimad.json"]
verano = ["201807_Usage_Bicimad.json", "201808_Usage_Bicimad.json","201809_Usage_Bicimad.json"]
otoño = ["201810_Usage_Bicimad.json", "201811_Usage_Bicimad.json","201812_Usage_Bicimad.json"]

#ahora crearemos la función trimestres, que tratará de transformar cada lista en un rdd con los datos de tres meses en cada uno de ellos
def trimestres(sc,lista_filenames): 
    lista_rdds = []
    for mes in lista_filenames:
        lista_rdds.append(sc.textFile(mes))#nos transformará cada archivo en un rdd y lo almacenará en una lista de ellos
    rdd_final = sc.union(lista_rdds)#después los uniremos en un solo rdd con datos sin distinguir a qué mes pertenecen
    rdd_final = rdd_final.map(mapper) #convertimos cada dato en tuplas
    #filtramos para solo estudiar al tipo 1 y agrupamos los datos de salida y entrada de las bicis y el tiempo de viaje en una tupla.
    rdd_final = rdd_final.filter(lambda x: x[0]==1).map(lambda x: (tuple(x[2:])))
    return rdd_final

#se lo aplicamos a cada estación para obtener los cuatro rdds representando los datos del año divididos por estaciones
rdd_inv = trimestres(sc,invierno).filter(lambda x: x[1]<300)
rdd_prim = trimestres(sc,primavera).filter(lambda x: x[1]<300)
rdd_ver = trimestres(sc,verano).filter(lambda x: x[1]<300)
rdd_ot = trimestres(sc,otoño).filter(lambda x: x[1]<300)
#hacemos el filter porque hemos observado un valor de una estación de bicis aislado, que es el 2008 y no nos interesa estudiarlo.

In [35]:
rdd_ot.take(1000)

[(54, 133, 312),
 (1, 59, 280),
 (64, 74, 276),
 (134, 127, 365),
 (123, 168, 410),
 (110, 94, 248),
 (6, 97, 582),
 (66, 103, 214),
 (21, 129, 323),
 (54, 136, 474),
 (161, 166, 403),
 (65, 96, 337),
 (81, 134, 589),
 (131, 174, 541),
 (52, 175, 593),
 (48, 56, 531),
 (92, 97, 112),
 (127, 55, 404),
 (175, 174, 97),
 (16, 56, 326),
 (1, 77, 700),
 (100, 48, 591),
 (67, 7, 646),
 (21, 129, 361),
 (21, 34, 836),
 (53, 74, 620),
 (8, 44, 660),
 (88, 103, 308),
 (109, 119, 547),
 (48, 43, 567),
 (33, 71, 137),
 (97, 161, 431),
 (154, 160, 450),
 (13, 149, 459),
 (92, 57, 545),
 (9, 175, 683),
 (10, 65, 626),
 (76, 67, 126),
 (83, 170, 762),
 (38, 57, 310),
 (161, 84, 735),
 (161, 84, 779),
 (1, 83, 559),
 (94, 4, 841),
 (92, 135, 1329),
 (66, 103, 112),
 (172, 114, 504),
 (92, 57, 740),
 (116, 173, 699),
 (74, 127, 637),
 (36, 49, 232),
 (106, 147, 860),
 (56, 27, 676),
 (82, 27, 1396),
 (145, 132, 1000),
 (17, 82, 826),
 (95, 166, 347),
 (57, 79, 495),
 (98, 138, 1135),
 (46, 81, 1430),


In [36]:
rdd_prueba = rdd_inv.filter(lambda x: x[1]>300)
rdd_prueba.take(20)

[]

In [42]:
#ahora ya tenemos cuatro rdds con los datos de tres meses cada uno unidos, usando union como en algunos ejemplos vistos.
#a cada uno de ellos le podremos aplicar nuestro programa de más arriba. En esta ocasión uniremos todos los pasos dentro de una misma función:

def mas_bicis_salida(rdd_util):
    #queremos contar el número de bicis que salen de cada estación:
    count_out = rdd_util.map(lambda x: x[0]).countByValue()
    rdd_out = sc.parallelize(list(count_out.items()))
    #las agruparemos por zonas,  10 estaciones por zona, utilizando la función ya definida previamente 'agrupar_estaciones': 
    rdd_out_agrup = rdd_out.map(agrupar_estaciones).reduceByKey(lambda x,y: x + y)
    #lo último que nos quedará será agrupar de mayor a menor este rdd:
    rdd_out_agrup_ord = rdd_out_agrup.sortBy(lambda x : x[1], ascending = False)
    return rdd_out_agrup_ord
    
def mas_bicis_entrada(rdd_util):
    #queremos contar el número de bicis que entran en cada estación
    count_in = rdd_util.map(lambda x: x[1]).countByValue()
    rdd_in = sc.parallelize(list(count_in.items()))
    #las agruparemos por zonas igual que en el caso anterior:
    rdd_in_agrup = rdd_in.map(agrupar_estaciones).reduceByKey(lambda x,y: x + y)
    #lo último que nos quedará será agrupar de mayor a menor este rdd:
    rdd_in_agrup_ord = rdd_in_agrup.sortBy(lambda x : x[1], ascending = False)
    return rdd_in_agrup_ord

In [43]:
#aplicamos cada función a los cuatro rdds, para ver en cada estación del año de qué zona salen más bicis y a qué zona entran:
salida_inv = mas_bicis_salida(rdd_inv)
salida_prim = mas_bicis_salida(rdd_prim)
salida_ver = mas_bicis_salida(rdd_ver)
salida_ot = mas_bicis_salida(rdd_ot)

entrada_inv = mas_bicis_entrada(rdd_inv)
entrada_prim = mas_bicis_entrada(rdd_prim)
entrada_ver = mas_bicis_entrada(rdd_ver)
entrada_ot = mas_bicis_entrada(rdd_ot)



In [61]:
salida_ot.take(20)

[(16, 78305),
 (5, 71127),
 (4, 58787),
 (13, 55428),
 (1, 51704),
 (0, 51244),
 (7, 49092),
 (9, 48697),
 (8, 47091),
 (12, 46785),
 (6, 45189),
 (3, 43370),
 (14, 41577),
 (11, 41335),
 (15, 41251),
 (10, 38212),
 (17, 33826),
 (2, 27147)]

In [65]:
entrada_ot.take(20)

[(16, 78647),
 (5, 71141),
 (13, 63516),
 (4, 61175),
 (1, 50997),
 (7, 50339),
 (8, 50291),
 (0, 49883),
 (12, 48485),
 (9, 46421),
 (6, 45218),
 (3, 42720),
 (11, 39228),
 (14, 38894),
 (15, 38211),
 (10, 34963),
 (17, 34249),
 (2, 25789)]

# Extra
Como tenemos los ficheros de datos agrupados de tres en tres meses, podemos aprovecharlo para ser capaces de observar la cantidad de bicicletas que "se mueven", es decir, son alquiladas en cada estación del año y, por tanto, observar en qué momentos del año la demanda es más alta.

Además, podemos comprobar si el número de bicicletas que salen en total son igual a las que entran, por si alguna bici se ha extraviado.

In [66]:
#lo último que haremos, aprovechando que tenemos todas las bicis que salen y que entran en total cada trimestre
#será ver cuál de todos los trimestres será en el que más bicis se alquilan.
num_inv = salida_inv.map(lambda x: x[1]).reduce(lambda x,y: x + y)
num_prim = salida_prim.map(lambda x: x[1]).reduce(lambda x,y: x + y)
num_ver = salida_ver.map(lambda x: x[1]).reduce(lambda x,y: x + y)
num_ot = salida_ot.map(lambda x: x[1]).reduce(lambda x,y: x + y)

In [67]:
num_inv, num_prim, num_ver, num_ot

(633200, 830476, 994035, 870167)

In [68]:
#también podríamos querer ver si se ha perdido alguna bicicleta viendo el número de ellas que entran a las estaciones
#de modo que si hay menos que las que salieron, es que hay alguna que no ha llegado nunca a ser devuelta.
num_inv2 = entrada_inv.map(lambda x: x[1]).reduce(lambda x,y: x + y)
num_prim2 = entrada_prim.map(lambda x: x[1]).reduce(lambda x,y: x + y)
num_ver2 = entrada_ver.map(lambda x: x[1]).reduce(lambda x,y: x + y)
num_ot2 = entrada_ot.map(lambda x: x[1]).reduce(lambda x,y: x + y)

In [69]:
num_inv2, num_prim2, num_ver2, num_ot2

(633200, 830476, 994035, 870167)